In [ ]:
from langchain_core.tools import tool

@tool
def calculate_sum(x: int, y: int) -> int:
    """Calculate the sum of two integers."""
    return x + y

@tool
def calculate_difference(x: int, y: int) -> int:
    """Calculate the difference of two integers."""
    return x - y
